# Artificial Neural Network
Constitutive law identification with Artificial Neural Network
(c) by Olivier Pantalé 2023

## Notes of version

# Initialization

In [ ]:
%run Common.ipynb
plotFigs = False
saveFigs = False

# Import Data from the HF5 file

Read Data from the HF5 file

In [ ]:
# Read the HF5 data file
DataFile = '3Cr2Mo'
data = readH5(filename = DataFile +'.h5')

depsArray = getDepsparray(data)
tempArray = getTarray(data)
numberOfData = len(data)
print('depsArray',depsArray,'contains',depsArray.shape[0],'data')
print('tempArray',tempArray,'contains',tempArray.shape[0],'data')
print('total of',numberOfData,'data')

Plot the content of the data to see what it looks like

In [ ]:
if (plotFigs):
    plotDatas(data)

Assemble all data into one big chunk

In [ ]:
removeZero = True # Remove the first point where eps = 0

first = True
for d in data:
    subdata = np.ones_like(d[2])*np.array([d[0],d[1]])
    newdata = np.array([d[2][:,0], subdata[:,0], subdata[:,1], d[2][:,1]])
    if first :
        allData = newdata
        first = False
    else:
        allData = np.concatenate((allData, newdata), axis=1)
print('Format of allData', allData.shape)
if (removeZero): 
    identData = allData[:,allData[0,:] != 0]
else:
    identData = allData
print('Format of identData', identData.shape)

Definition of the non classic functions and their derivatives

In [ ]:
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

def dsigmoid(x):
    return (sigmoid(x)*(1-sigmoid(x)))
    #return np.exp(-x) / (1 + np.exp(-x))**2
    #return 1 / (np.exp(x) * (1+np.exp(-x))**2)

def dtanh(x):
    return 1 - np.tanh(x)**2
    
def softplus(x):
    return np.log1p(np.exp(x))
    #return np.log1p(np.exp(-np.abs(x))) + np.maximum(x, 0)
    
def dsoftplus(x):
    return sigmoid(x)

def swish(x):
    return x * sigmoid(x)

def dswish(x):
    return swish(x) + sigmoid(x)*(1 - swish(x)) 

def relu(x):
    return x*(x>0)

def drelu(x):
    return 1*(x>0)

In [ ]:
variableFormatToUse = "%.16f"

def valueString(a):
    s = variableFormatToUse%(a)
    for c in range(len(s)-1,0,-1):
        if (s[c] != '0'): break
    if (s[c]=='.'): return s[0:c]
    return s[0:c+1]

comment = 'C '
cline = 'C **********************************************************************\n'
block =   '      '

def splitFortranLine(line):
    line = line.replace('|*','\n     + *')
    line = line.replace('|/','\n     + /')
    line = line.replace('| + ','\n     + +')
    line = line.replace('| - ','\n     + -')
    return line

def toPython(line):
    if (line[0] == 'C'): line='#'+line[1:]
    line = line.replace('|','')
    line = line.replace('then',':')
    line = line.replace('endif','# endif')
    line = line.replace('else','else:')    
    line = line.replace('exp','np.exp')
    line = line.replace('log','np.log')
    line = line.replace('tanh','np.tanh')
    line = line.replace('abs','np.abs')
    line = line.replace("--", "+")
    line = line.replace("+-", "-")
    line = line.replace("(+", "(")
    line = line.replace('+',' + ')
    line = line.replace('-',' - ')
    line = line.replace('=  - ','=-')
    line = line.replace('( - ','(-')
    line = line.replace('  +  ',' + ')
    #line = line.replace('.000000000000','.0')
    line = line.replace('(k)','')
    line = line.replace('(k,1)','1')
    line = line.replace('(k,2)','2')
    line = line + '\n'
    return line

def toFortran(line):
    line = line.replace("--", "+")
    line = line.replace("max", "dmax1")
    line = line.replace("+-", "-")
    line = line.replace("(+", "(")
    line = line.replace('+',' + ')
    line = line.replace('-',' - ')
    line = line.replace('=  - ','=-')
    line = line.replace('( - ','(-')
    line = line.replace('  +  ',' + ')
    line = line.replace('.000000000000','.0')
    if (line[0] == 'C'): line = line + '\n'
    else: line = block + line + '\n'
    line = splitFortranLine(line)
    return line

def dataZone(code):
    code = 'C Block of Data\n'
    code += writeDataFortran(w1, 'w1')
    code += writeDataFortran(b1, 'b1')
    code += writeDataFortran(w2, 'w2')
    code += writeDataFortran(b2, 'b2')
    code += writeDataFortran(w3, 'w3')
    code += writeDataFortran(b3, 'b3')
    code += writeDataFortran(minInput, 'xmI')
    code += writeDataFortran(rangeInput, 'xrI')
    code += writeDataFortran(np.array([[minOutput[0]]]), 'xmO')
    code += writeDataFortran(np.array([[rangeOutput[0]]]), 'xrO')
    code += writeDataFortran(np.array([[deps0]]), 'xdeps0')
    return code

def uhardHead(data = False):
    code = cline
    code += comment + 'Function to compute the ANN : ' + DataFile + '-' + modelName + ' yield stress\n'
    code += cline
    code += '      subroutine uhard (\n'
    code += '     +  syield, hard, eqplas, eqplasrt, time, dtime, temp, \n'
    code += '     +  dtemp, noel, npt, layer, kspt, kstep, kinc, \n'
    code += '     +  cmname, nstatv, statev, numfieldv, \n'
    code += '     +  predef, dpredef, numprops, props)\n'
    code += 'C\n'
    code += "      include 'aba_param.inc'\n"
    code += 'C\n'
    code += '      dimension hard(3), statev(nstatv), time(*),\n'
    code += '     +  predef(numfieldv), dpredef(*), props(*)\n'
    code += 'C\n'
    code += '      character*80 cmname\n'
    if (data):
        code += dataZone(code)
    return code

def vuhardHead(data = False):
    code = cline
    code += comment + 'Function to compute the ANN : ' + DataFile + '-' + modelName + ' yield stress\n'
    code += cline
    code += '      subroutine vuhard (\n'
    code += 'C Read only -\n'
    code += '     +  nblock, nElement, nIntPt, nLayer, nSecPt, lAnneal, stepTime,\n'
    code += '     +  totalTime, dt, cmname, nstatev, nfieldv, nprops, props,\n'
    code += '     +  tempOld, tempNew, fieldOld, fieldNew, stateOld, eqps, eqpsRate,\n'
    code += 'C Write only -\n'
    code += '     +  yield, dyieldDtemp, dyieldDeqps, stateNew)\n'
    code += 'C\n'
    code += "      include 'vaba_param.inc'\n"
    code += 'C\n'
    code += '      dimension nElement(nblock), props(nprops), tempOld(nblock),\n'
    code += '     +  fieldOld(nblock,nfieldv), stateOld(nblock,nstatev),\n'
    code += '     +  tempNew(nblock), fieldNew(nblock,nfieldv), eqps(nblock),\n'
    code += '     +  eqpsRate(nblock), yield(nblock), dyieldDtemp(nblock),\n'
    code += '     +  dyieldDeqps(nblock,2), stateNew(nblock,nstatev)\n'
    code += 'C\n'
    code += '      character*80 cmname\n'
    if (data):
        code += dataZone(code)
    code += 'C Do the main loop for all block values\n'
    code += '      do k = 1, nblock\n'
    return code

def vuhardTail():
    code = 'C Store the eqpsRate into stateNew variable 1\n'
    code += '      stateNew(k,1) = eqpsRate(k)\n'
    code += '      end do\n'
    code += 'C Return from the VUHARD subroutine\n'
    code += '      return\n'
    code += '      end\n'
    return code

def uhardTail():
    code = 'C Store the eqplasrt into statev variable 1\n'
    code += '      statev(1) = eqplasrt\n'
    code += 'C Return from the UHARD subroutine\n'
    code += '      return\n'
    code += '      end\n'
    return code
    
def encodeFunction(f, p, func):
    f += toFortran(func)
    p += toPython(func)
    return f, p

def writeDataFortran(var, name):
    I, J = var.shape
    lines = ''
    if (I == 1) and (J==1):
        lines += toFortran('double precision '+name)
    elif (I == 1) or (J==1):
        if (I == 1):
            lines += toFortran('double precision '+name+'('+str(J)+')')
        if (J == 1):
            lines += toFortran('double precision '+name+'('+str(I)+')')
    else:
        lines += toFortran('double precision '+name+'('+str(I)+', '+str(J)+')')
    line = '      data '+name+'/'
    d = 0
    for j in range (J):
        for i in range (I):
            if (d>0): line += ',\n     + '
            line += valueString(var[i,j])+'D0'
            d += 1
    line += '/\n'        
    lines += line
    return lines

def wv1(n,i):
    return n+'('+str(i+1)+')'
    
def wv2(n,i,j):
    return n+'('+str(i+1)+','+str(j+1)+')'

def wv(i):
    return str(i+1)    

Load the ANN parameters of the model to test

In [ ]:
activations = [['sigmoid', sigmoid, dsigmoid],
               ['softplus', softplus, dsoftplus],
               ['tanh', np.tanh, dtanh],
               ['swish', swish, dswish],
               ['relu', relu, drelu],
               ['exponential', np.exp, np.exp]]

take = 5 # Define the model to export [0,5]
shape = [17,9]

activation = activations[take][0]
actF = activations[take][1]
dactF = activations[take][2]

def getModelName(shape, activation):
    return '3-' + str(shape[0]) + '-' + str(shape[1]) + '-1-' + activation

# Get model name
modelName = getModelName(shape, activation) 

print('Selected model is:', modelName)

# Read and unpack data
W, B, R = readAnnH5(DataFile + '/' + modelName + '-ANN.h5')
w1, w2, w3 = W
b1, b2, b3 = B
minData, maxData, deps0 = R

minInput = minData[0:3,0,None]
maxInput = maxData[0:3,0,None]
minOutput = minData[3]
maxOutput = maxData[3]
rangeInput = maxInput - minInput
rangeOutput = maxOutput - minOutput

print('Inputs : min=',minInput.T,' max=', maxInput.T)
print('Output : min=',minOutput.T,' max=', maxOutput.T)

Split Data into annInput and annOutput and display the shapes

In [ ]:
annInput = np.copy(identData[0:3,:])
annOutput = np.copy(identData[None,3,:])
print(annInput.shape, annOutput.shape)

Definition of the structure of the ANN and the computation of the derivatives

In [ ]:
def AnnFunctionWithDerivatives(input, activation, dactivation):
    x = np.copy(input)
    x[1,:] = np.log(x[1,:] / deps0)
    x = (x - minInput) / rangeInput
    y1 = w1.dot(x) + b1
    f1 = activation(y1)
    y2 = w2.dot(f1) + b2
    f2 = activation(y2)
    sigma = (w3.dot(f2) + b3) * rangeOutput + minOutput
    dsigma = (w1.T).dot((w2.T).dot(w3.T * dactivation(y2)) * dactivation(y1)) * rangeOutput / rangeInput
    dsigma[1,:] = dsigma[1,:] / input[1,:]
    return sigma, dsigma 

Test by computing sigma and its derivatives

In [ ]:
sig, dsig = AnnFunctionWithDerivatives(annInput, actF, dactF)
error = np.abs((sig-annOutput)/annOutput)
print("Mean error is :", error.mean())

## Generation of the Fortran and Python code
### Definition of activations functions to use

Activation functions:

Sigmoid :
$$\sigma = \frac{1}{1+\exp(-x)}$$
$$\sigma' = \sigma\left(1-\sigma\right)$$

Exponential:
$$f = \exp(x)$$
$$f' = f$$

In [ ]:
def actFortran(act, x, i):
    exist = False
    if (act == 'sigmoid'):
        exist = True
        fct = '1/(1+exp(-X))'
    if (act == 'exponential'):
        exist = True
        fct = 'exp(X)'
    if (act == 'tanh'):
        exist = True
        fct = 'tanh(X)'
    if (act == 'swish'):
        exist = True
        fct = 'X/(1+exp(-X))'
    if (act == 'softplus'):
        exist = True
        #fct = 'log(1+exp(-abs(X)))+max(0.0D0,X)'
        fct = 'log(1+exp(X))'
        # np.log1p(np.exp(-np.abs(x))) + np.maximum(x, 0)
    if (act == 'relu'):
        exist = True
        fct = '(max(0.0D0,X))'
        # np.log1p(np.exp(-np.abs(x))) + np.maximum(x, 0)
    if (not exist):
        print('ERROR Its not implemented yet')
        return '',''
    return fct.replace('X',x).replace('#',str(i))

def dactFortran(act, y, x, i):
    exist = False
    if (act == 'sigmoid'):
        exist = True
        fct = '(Y*(1-Y))' # sigmoid(x)*(1-sigmoid(x)) = y*(1-y)
    if (act == 'exponential'):
        exist = True
        fct = 'Y' # exp(x) = y
    if (act == 'tanh'):
        exist = True
        fct = '(1-Y*Y)' # 1-tanh(x)**2 = 1-y*y
    if (act == 'swish'):
        exist = True
        fct = '(Y+(1-Y)/(1+exp(-X)))' # swish(x) + sigmoid(x)*(1 - swish(x)) = y + sigmoid(x)*(1-y)
    if (act == 'softplus'):
        exist = True
        fct = '(1/(1+exp(-X)))' # swish(x) + sigmoid(x)*(1 - swish(x)) = y + sigmoid(x)*(1-y)
    if (act == 'relu'):
        exist = True
        fct = 'max(0.0D0,sign(1.0D0,X))'
    if (not exist):
        print('ERROR Its not implemented yet')
        return '',''
    return fct.replace('X',x).replace('Y',y).replace('#',str(i))

In [ ]:
#max(0, np.sign(1,2))

### Initialization

In [ ]:
fortranCode = ''
pythonCode = ''
inlineFactor = True
useData = True

Explicit = False
Explicit = True

### Write the header part of the Fortran code

In [ ]:
if (Explicit):
    fortranCode += vuhardHead(useData)
else:
    fortranCode += uhardHead(useData)

In [ ]:
#print(fortranCode)

### Preprocessing of the variables

In [ ]:
# Write comment
fortranCode, pythonCode = encodeFunction(fortranCode, pythonCode, comment + 'Preprocessing of the variables')

if (Explicit):
    var_xeps = 'eqps(k)'
    var_xdeps = 'eqpsRate(k)'
    var_xtemp = 'tempNew(k)'
else:
    var_xeps = 'eqplas'
    var_xdeps = 'eqplasrt'
    var_xtemp = 'temp+dtemp'
    
# Compute for eps
if (useData):
    fortranCode, pythonCode = encodeFunction(fortranCode, pythonCode, 'xeps = ('+var_xeps+'-xmI(1))/xrI(1)')
else:
    fortranCode, pythonCode = encodeFunction(fortranCode, pythonCode, 'xeps = ('+var_xeps+'-' + valueString(minInput[0,0]) + ')/' + valueString(rangeInput[0,0]))

# Compute for deps
if (useData):
    fortranCode, pythonCode = encodeFunction(fortranCode, pythonCode, 'if ('+var_xdeps+' > xdeps0) then')
    fortranCode, pythonCode = encodeFunction(fortranCode, pythonCode, '  xdeps = (log('+var_xdeps+'/xdeps0)-xmI(2))/xrI(2)')
    fortranCode, pythonCode = encodeFunction(fortranCode, pythonCode, 'else')
    fortranCode, pythonCode = encodeFunction(fortranCode, pythonCode, '  xdeps = 0')
    fortranCode, pythonCode = encodeFunction(fortranCode, pythonCode, '  '+var_xdeps+' = xdeps0')
    fortranCode, pythonCode = encodeFunction(fortranCode, pythonCode, 'endif')
else:
    fortranCode, pythonCode = encodeFunction(fortranCode, pythonCode, 'if ('+var_xdeps+' > ' + valueString(deps0) + ') then')
    fortranCode, pythonCode = encodeFunction(fortranCode, pythonCode, '  xdeps = (log('+var_xdeps+'/' + valueString(deps0) + ')-' + valueString(minInput[1,0]) + ')/' + valueString(rangeInput[1,0]))
    fortranCode, pythonCode = encodeFunction(fortranCode, pythonCode, 'else')
    fortranCode, pythonCode = encodeFunction(fortranCode, pythonCode, '  xdeps = 0')
    fortranCode, pythonCode = encodeFunction(fortranCode, pythonCode, '  '+var_xdeps+' = ' + valueString(deps0))
    fortranCode, pythonCode = encodeFunction(fortranCode, pythonCode, 'endif')

# Compute for T
if (useData):
    fortranCode, pythonCode = encodeFunction(fortranCode, pythonCode, 'xtemp = ('+var_xtemp+'-xmI(3))/xrI(3)')
else:
    fortranCode, pythonCode = encodeFunction(fortranCode, pythonCode, 'xtemp = ('+var_xtemp+'-' + valueString(minInput[2,0])+')/' + valueString(rangeInput[2,0]))

### Core of the Artificial Neural Network

In [ ]:
# Write the comment for the function
fortranCode, pythonCode = encodeFunction(fortranCode, pythonCode, comment + 'Hidden layer #1-(y11 to y1'+str(b1.shape[0])+')')
    
# Write the function
for i in range(b1.shape[0]):
    if (useData):
        fortranCode, pythonCode = encodeFunction(fortranCode, pythonCode, 'y1' + wv(i) + ' = ' + wv2('w1',i,0) + ' * xeps|+' + wv2('w1',i,1) + ' * xdeps|+' + wv2('w1',i,2) + ' * xtemp|+' + wv1('b1',i))
    else:
        fortranCode, pythonCode = encodeFunction(fortranCode, pythonCode, 'y1' + wv(i) + ' = ' + valueString(w1[i,0]) + ' * xeps|+'+ valueString(w1[i,1]) + ' * xdeps|+'+ valueString(w1[i,2]) + ' * xtemp|+' + valueString(b1[i,0]))

In [ ]:
fortranCode, pythonCode = encodeFunction(fortranCode, pythonCode, comment + activation + ' activation function-(yf11 to yf1'+str(b1.shape[0])+')')

for i in range(b1.shape[0]):
    fortranCode, pythonCode = encodeFunction(fortranCode, pythonCode, 'yf1' + wv(i) + ' = ' + actFortran(activation, 'y1#', i+1))

In [ ]:
# Write the comment for the function
fortranCode, pythonCode = encodeFunction(fortranCode, pythonCode, comment + 'Hidden layer #2-(y21 to y2'+str(b2.shape[0])+')')

# Write the function
if (useData):
    for i in range(b2.shape[0]):
        line = 'y2' + wv(i) + " = "
        for j in range(w2.shape[1]):
            if (j != 0): line += '+'
            line += wv2('w2',i,j) + ' * yf1' + wv(j) + '|'
        line += '+' + wv1('b2',i)
        fortranCode, pythonCode = encodeFunction(fortranCode, pythonCode, line)
else:
    for i in range(b2.shape[0]):
        line = 'y2' + wv(i) + " = "
        for j in range(w2.shape[1]):
            line += '+' + valueString(w2[i,j]) + ' * yf1' + wv(j) + '|'
        line += '+' + valueString(b2[i,0])
        fortranCode, pythonCode = encodeFunction(fortranCode, pythonCode, line)

In [ ]:
fortranCode, pythonCode = encodeFunction(fortranCode, pythonCode, comment + activation + ' activation function-(yf21 to yf2'+str(b2.shape[0])+')')

for i in range(b2.shape[0]):
    fortranCode, pythonCode = encodeFunction(fortranCode, pythonCode, 'yf2' + wv(i) + ' = ' + actFortran(activation, 'y2#', i+1))

In [ ]:
# Write the comment for the function
fortranCode, pythonCode = encodeFunction(fortranCode, pythonCode, comment + 'Derivatives terms-(xa1 to xa'+str(w3.shape[1])+') and (xb1 to xb'+str(w2.shape[1])+')')

# Write the function (yf2*(1-yf2))
# xa = w3.T * dactivation(y2))
if (useData):
    for i in range(w3.shape[1]):
        fortranCode, pythonCode = encodeFunction(fortranCode, pythonCode, 'xa' + wv(i) + ' = ' + wv1('w3',i) + ' * ' + dactFortran(activation,'yf2#','y2#',i+1))
else:
    for i in range(w3.shape[1]):
        fortranCode, pythonCode = encodeFunction(fortranCode, pythonCode, 'xa' + wv(i) + ' = ' + valueString(w3[0,i]) + ' * ' + dactFortran(activation,'yf2#','y2#',i+1))

# xb = (w2.T).dot(xa * dactivation(y1)
if (useData):    
    for i in range(w2.shape[1]):
        line = 'xb' + wv(i) + ' = ('
        for j in range(w2.shape[0]):
            if (j!=0): line += '|+'
            line += wv2('w2',j,i) + ' * ' + 'xa' + wv(j)
        line += ')|* ' + dactFortran(activation,'yf1#','y1#',i+1) #yf1' + wv(i) + '*(1-yf1' + wv(i) + ')'
        fortranCode, pythonCode = encodeFunction(fortranCode, pythonCode, line)
else:
    for i in range(w2.shape[1]):
        line = 'xb' + wv(i) + ' = ('
        for j in range(w2.shape[0]):
            if (j!=0): line += '|+'
            line += valueString(w2[j,i]) + ' * ' + 'xa' + wv(j)
        line += ')|* ' + dactFortran(activation,'yf1#','y1#',i+1) #yf1' + wv(i) + '*(1-yf1' + wv(i) + ')'
        fortranCode, pythonCode = encodeFunction(fortranCode, pythonCode, line)

In [ ]:
if (Explicit):
    var_sig = 'Yield(k)'
    var_dsig1 = 'dyieldDeqps(k,1)'
    var_dsig2 = 'dyieldDeqps(k,1)'
    var_dsig3 = 'dyieldDtemp(k)'
else:
    var_sig = 'syield'
    var_dsig1 = 'hard(1)'
    var_dsig2 = 'hard(2)'
    var_dsig3 = 'hard(3)'
    
# Write the comment for the function
fortranCode, pythonCode = encodeFunction(fortranCode, pythonCode, comment + 'Outputs of the subroutine')

# Write the function
if (useData):    
    line = var_sig + ' = xrO * ('
    for i in range(w3.shape[1]):
        if (i != 0): line += '|+'
        line += wv1('w3',i) + ' * yf2' + wv(i)
    line += '|+b3)|+xmO' 
    fortranCode, pythonCode = encodeFunction(fortranCode, pythonCode, line)
else:
    if (inlineFactor):
        line = var_sig + ' = '
        for i in range(w3.shape[1]):
            if (i != 0): line += '|+'
            line += valueString(rangeOutput[0] * w3[0,i]) + ' * yf2' + wv(i)
        line += '|+' + valueString(rangeOutput[0] * b3[0][0] + minOutput[0]) 
        fortranCode, pythonCode = encodeFunction(fortranCode, pythonCode, line)
    else:
        line = var_sig + ' = ' + valueString(rangeOutput[0]) + '|*('
        for i in range(w3.shape[1]):
            if (i != 0): line += '|+'
            line += valueString(w3[0,i]) + ' * yf2' + wv(i)
        line += '|+' + valueString(b3[0][0]) + ')|+' + valueString(minOutput[0]) 
        fortranCode, pythonCode = encodeFunction(fortranCode, pythonCode, line)
    
# Write the derivatives of the functions
if (useData):   
    line = var_dsig1 + ' = xrO * ('
    for j in range(w1.shape[0]):
        if (j != 0): line += '|+'
        line += wv2('w1',j,0) + ' * ' + 'xb' + wv(j)
    line += ') / xrI(1)'
    fortranCode, pythonCode = encodeFunction(fortranCode, pythonCode, line)
    line = var_dsig2 + ' = xrO * ('
    for j in range(w1.shape[0]):
        if (j != 0): line += '|+'
        line += wv2('w1',j,1) + ' * ' + 'xb' + wv(j)
    line += ')|/(xrI(2)*'+var_xdeps+')'
    fortranCode, pythonCode = encodeFunction(fortranCode, pythonCode, line)
    line = var_dsig3 + ' = xrO * ('
    for j in range(w1.shape[0]):
        if (j != 0): line += '|+'
        line += wv2('w1',j,2) + ' * ' + 'xb' + wv(j)
    line += ') / xrI(3)'
    fortranCode, pythonCode = encodeFunction(fortranCode, pythonCode, line)
else:
    if (inlineFactor):
        line = var_dsig1 + ' = '
        for j in range(w1.shape[0]):
            if (j != 0): line += '|+'
            line += valueString(rangeOutput[0]*w1[j,0]/rangeInput[0,0]) + ' * ' + 'xb' + wv(j)
        fortranCode, pythonCode = encodeFunction(fortranCode, pythonCode, line)
        line = var_dsig2 + ' = ('
        for j in range(w1.shape[0]):
            if (j != 0): line += '|+'
            line += valueString(rangeOutput[0]*w1[j,1]/rangeInput[1,0]) + ' * ' + 'xb' + wv(j)
        line += ') / '+var_xdeps+''
        fortranCode, pythonCode = encodeFunction(fortranCode, pythonCode, line)
        line = var_dsig3 + ' = '
        for j in range(w1.shape[0]):
            if (j != 0): line += '|+'
            line += valueString(rangeOutput[0]*w1[j,2]/rangeInput[2,0]) + ' * ' + 'xb' + wv(j)
        fortranCode, pythonCode = encodeFunction(fortranCode, pythonCode, line)
    else:
        line = var_dsig1 + ' = ' + valueString(rangeOutput[0]) + '|* ('
        for j in range(w1.shape[0]):
            if (j != 0): line += '|+'
            line += valueString(w1[j,0]) + ' * ' + 'xb' + wv(j)
        line += ') / ' + valueString(rangeInput[0,0])
        fortranCode, pythonCode = encodeFunction(fortranCode, pythonCode, line)
        line = var_dsig2 + ' = ' + valueString(rangeOutput[0]) + '|* ('
        for j in range(w1.shape[0]):
            if (j != 0): line += '|+'
            line += valueString(w1[j,1]) + ' * ' + 'xb' + wv(j)
        line += ')|/(' + valueString(rangeInput[1,0]) + '*'+var_xdeps+')'
        fortranCode, pythonCode = encodeFunction(fortranCode, pythonCode, line)
        line = var_dsig3 + ' = ' + valueString(rangeOutput[0]) + '|* ('
        for j in range(w1.shape[0]):
            if (j != 0): line += '|+'
            line += valueString(w1[j,2]) + ' * ' + 'xb' + wv(j)
        line += ') / ' + valueString(rangeInput[2,0])
        fortranCode, pythonCode = encodeFunction(fortranCode, pythonCode, line)

In [ ]:
if (Explicit):
    fortranCode += vuhardTail()
else:
    fortranCode += uhardTail()    

In [ ]:
print(fortranCode)

In [ ]:
testVal = 101
eqps = annInput[0,testVal]
eqpsRate = annInput[1,testVal]
tempNew = annInput[2,testVal]
print(eqps, eqpsRate, tempNew)

## Writes the fortran file

In [ ]:
text_file = open(DataFile + '-' + modelName + ".f", "w")
text_file.write(fortranCode)
text_file.close()

## Test the execution

In [ ]:
# Execute the corresponding Python code to test
if (not useData):
    exec(pythonCode)
    print('Yield stress',Yield)
    Dsigma=np.array([[dyieldDeqps1, dyieldDeqps2, dyieldDtemp]])
    print('Yield stress derivatives',Dsigma)   
    x = np.copy(annInput[None,:,testVal].T)
    deps = np.copy(x[1])
    #print("x=", x.T)
    x[1,:] = np.log(x[1,:] / deps0)
    x = (x - minInput) / rangeInput
    #print("x=",x.T)
    y1 = w1.dot(x) + b1
    #print("y1=",y1.T)
    f1 = actF(y1)
    #print("f1=",f1.T)
    y2 = w2.dot(f1) + b2
    #print("y2=",y2.T)
    f2 = actF(y2)
    #print("f2=",f2.T)
    sigma = (w3.dot(f2) + b3) * rangeOutput + minOutput
    print('Yield stress',sigma)
    dsigma = (w1.T).dot((w2.T).dot(w3.T * dactF(y2)) * dactF(y1)) * rangeOutput / rangeInput
    #print("dsigma=",dsigma)
    dsigma[1,:] = dsigma[1,:] / deps
    print('Yield stress derivatives',dsigma.T)
    print('Yield stress error', sigma - Yield)
    print('Yield stress derivatives error', dsigma.T - Dsigma)

## Writes the latex File

In [ ]:
def np2lat(f, n, A, prec=4):
    if (A.ndim == 1):
        cols = 1
    else:
        cols = A.shape[1]
    tabformat = '%.'+str(prec)+'f'
    tabalign = 'r'*cols
    f.write('\\begin{equation*}\n')
    f.write(n+' = ')
    f.write('\\left[\n')
    f.write('\\begin{array}{%s}\n' %tabalign)
    np.savetxt(f, A, fmt=tabformat, delimiter=' & ', newline='\\\\ \n')
    f.write('\\end{array}\\right]\n')
    f.write('\\end{equation*}\n')

def writeLatex(filename, prec=4):
    f = open(filename, 'w')
    np2lat(f, '\\w_1',w1, prec)
    np2lat(f,'\\overrightarrow{b}_1',b1, prec)
    np2lat(f, '\\w_2^T',w2.T, prec)
    np2lat(f,'\\overrightarrow{b}_2',b2, prec)
    np2lat(f, '\\overrightarrow{w}_3',w3.T, prec)
    np2lat(f,'b_3',b3, prec)
    np2lat(f, 'logBase',np.array([deps0]), prec)
    np2lat(f, 'minInput',minInput, prec)
    np2lat(f, 'rangeInput',rangeInput, prec)
    np2lat(f, 'minOutput',minOutput, prec)
    np2lat(f, 'rangeOutput',rangeOutput, prec)    
    f.close()

In [ ]:
writeLatex(DataFile + '-' + modelName + ".tex", prec=3)